# Plotly-dash
[참고] https://dash.plot.ly/  <br>
[참고2] https://dash.plot.ly/dash-core-components <br>
[참고3] https://dash.plot.ly/interactive-graphing

## Part2. Dash Layout
* plotly dash 실행 서버
    * http://127.0.0.1:8050/

> Dash구조
* layout은 html.Div 와 dcc.Graph로 구성되어 있음.
* dash_html_components 라이브러리는 모든 HTML태그의 컴포넌트가 들어있다. 
* dash_core_components 는 인터렉티브한 하이레벨의 컴포넌트를 제공. React.js 라이브러리기반
* children 프로퍼티는 특별하다. 
    * html.H1(children='Hello Dash') 와 html.H1('Hello Dash') 은 같다.

> 구성요소
* app.run_server(debug=True, use_reloader=False) 은 사용자가 코드를 변경했을 때 자동으로 Dash를 refresh 해주는 기능

> Visualization에 대한
* dash_core_components라이브러리는 Graph라 불리는 컴포넌트를 갖고 있다.
    * Graphh 는 plotly.js 오픈소스를 사용해서 인터렉티브한 데이터 시각화를 렌더해준다 .
* figure 는 plotly.py에서 사용되는 figure와 같은 기능. 

> Markdown
* dash_core_components 라이브러리의 Markdown 컴포넌트 사용. <br>
markdown_text = ''' <br>
마크다운 내용 입력 <br>
'''

## Part3. Basic Dash Callbacks

> Dash Layout
* app.callback 을 통해서 input과 output 가 표현된다.
* Dash에서 input과 output은 특정 컴포넌트의 속성이다. 
* conponent_id 와 component_property 키워드는 옵션이다.

    => 새로운 텍스트를 띄우기 위해서는 children을 업데이트하고, 새로운 데이터를 띄우기 위해서는 dcc.Graph 의 figure를 업데이트한다.


### [ error사항 해결방법 ]
* 아래처럼 use_reloader=False 값 추가해야됨.

if __name__ == '__main__':  <br>
    app.run_server(debug=True, use_reloader=False)<br>
    
    
    

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
import scipy as sp
import re

In [4]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

### 데이터 불러오기

In [8]:
# 날짜 데이터 불러올 때 
df=pd.read_excel('./data/raw_data_세부.xlsx',encoding='euc-kr',parse_dates={'날짜':["년","월"]})
df=df.rename(columns={'날짜':'date','분류1':'device','분류2':'shopping_type','서비스분류1':'service','종류1':'type1','종류2':'type2','값':'value'})

# 종합_쇼핑기여 제외 (종합 만 )
df=df[df['shopping_type']=='종합']
#df.head(10)

In [9]:
# 광고 타입 (직접광고, 네트워크 ...) 별 매출
## 서비스 전체매출
data_adtype_sales=df.query('type2!=["PV","UV","일평균방문횟수"] and device=="PC" and type1 == "매출 구분"')
data_adtype_sales=data_adtype_sales.dropna(subset=['value'])        # 'value'컬럼에 NA있을때 제거

# service컬럼에 '뉴스' 또는 '기사박스' 일경우 '뉴스'로 통일. 
## TV,스푼피드,여행,이글루스,자동차,증권정보,코인,허브,이슈트렌드는 모두 '서브'로 통일
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['기사박스']),'뉴스',data_adtype_sales['service'] )
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_adtype_sales['service_group'])
#print(data_adtype_sales['service_group'].unique())

In [10]:
# 광고구분없이 서비스별(프런트, 검색, 뉴스, 서브) 매출 (groupby sum)
data_adtype_sales_groupsum=pd.DataFrame(data_adtype_sales.groupby(['date','service_group'], as_index=False)['value'].sum()) 

In [11]:
# PC, 서비스별 트래픽(PV,UV) 데이터
data_traffic=df.query('device=="PC" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]
data_traffic['service_group']=np.where(data_traffic['service'].isin(['기사박스']),'뉴스',data_traffic['service'])
data_traffic['service_group']=np.where(data_traffic['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic['service_group'])

In [12]:
data_traffic_groupsum=pd.DataFrame(data_traffic.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

In [13]:
sales =data_adtype_sales.groupby(['date','service_group']).sum()
sales_pcts=pd.DataFrame(round((sales/sales.groupby(level=0).sum())*100,2).reset_index()['value'])
data_adtype_sales_groupsum=pd.concat([data_adtype_sales_groupsum, sales_pcts],axis=1)
data_adtype_sales_groupsum.columns=['date','service_group','value','pcts']

In [14]:
data_adtype_sales_groupsum['inc_pcts']=round(((sales.groupby(level=1).pct_change())*100).reset_index()['value'],2)

In [15]:
# 서비스별, 광고종류별 매출
### bar-plot으로 총매출 그리고, line-plot으로 직접광고, 쇼핑, 네트워크, 제휴 각각 그리기
### 프런트, 검색, 뉴스, 서브 각각 다른 subplot 4개로
data_adtype_sales_groupsum2=pd.DataFrame(data_adtype_sales.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

In [16]:
# 프론트 트래픽
data_traffic_front=df.query('device=="PC" and service=="프런트" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]

In [17]:
#  PC, 트래픽 평균단가
data_traffic_price=df.query('device=="PC" and type1=="트래픽 평균단가"')
data_traffic_price= data_traffic_price.dropna(subset=['value'])

# 서비스 카테고리 4개로 축소 : 프런트, 검색, 뉴스, 서브
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['기사박스']),'뉴스', data_traffic_price['service'])
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic_price['service_group'])

# 소수점 
data_traffic_price['value']=round(data_traffic_price['value'],2)

In [18]:
data_traffic_price_groupsum=pd.DataFrame(data_traffic_price.groupby(['date','service_group','type2'])['value'].sum().reset_index())

----------------------------------
## Dash-plot
#### PC-서비스별-트래픽-매출

In [21]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='KPI-Dash'),

    html.Div(children='''
        Dash: zum traffic and sales visualization
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '프런트_PV'},
                
                  {'x': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '검색_PV'},
                
                {'x': list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="뉴스"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '뉴스_PV'},
                
                {'x': list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="서브"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '서브_PV'},
                
                ### 매출
                {'x': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['value']), 
                 'type': 'bar', 
                 'name': '프런트_매출'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="검색"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="검색"')['value']), 
                 'type': 'bar', 
                 'name': '검색_매출'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="뉴스"')['value']), 
                 'type': 'bar', 
                 'name': '뉴스_매출'},
                                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="서브"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="서브"')['value']), 
                 'type': 'bar', 
                 'name': '서브_매출'},
                
            ],
            'layout': {
                'title': 'PC-서비스별-트래픽-매출'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 646-371-916
Debugger PIN: 646-371-916
Debugger PIN: 646-371-916
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


### Tab 기능 구현
[참고] https://dash.plot.ly/dash-core-components

In [22]:
import dash
import dash_html_components as html
import dash_core_components as dcc

from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Tab one', value='tab-1'),
        dcc.Tab(label='Tab two', value='tab-2'),
    ]),
    html.Div(id='tabs-content')
])

@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            html.H3('Tab content 1')
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])


if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 151-868-202
Debugger PIN: 151-868-202
Debugger PIN: 151-868-202
Debugger PIN: 151-868-202
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
